<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/08b_BERTClfFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a predecir el sentimiento de reviews de películas usando dos enfoques.

1. **BERT + fine-tuning**: fine-tuneamos BERT + una capa de clasifición lineal en el dataset de reviews.
2. **BERT "pre-fine-tuneado"**: usamos un modelo (BERT + clasificador) previamente entrenado para análisis de sentimiento, sin entrenar en nuestros datos.

-----------------------

Tarea: responder donde dice **PREGUNTA**

### Configuración del entorno


In [ ]:
!pip install -qU transformers accelerate datasets watermark

In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp transformers,datasets,torch,numpy,pandas,tqdm

Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU"

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Dataset

Cargamos y exploramos el dataset de reviews de películas de imdb.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")

In [ ]:
dataset

In [ ]:
import pandas as pd
import numpy as np
import datasets
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    """Muestra num_examples ejemplos aleatorios del dataset.
    """
    indices = np.random.randint(0, len(dataset), num_examples)
    df = pd.DataFrame(dataset[indices])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

np.random.seed(33)
show_random_elements(dataset["train"], num_examples=6)

In [ ]:
print("Distribucion de clases:")
for k in dataset.keys():
    print(k)
    print(pd.Series(dataset[k]["label"]).value_counts())
    print("-"*70)

In [ ]:
print("Largo de los documentos (en palabras), deciles:")
for k in dataset.keys():
    print(k)
    largos = pd.Series(dataset[k]["text"]).str.split().apply(len)
    print(np.quantile(largos, q=np.arange(0, 1.1, .1)).astype(int))
    print("-"*70)

In [ ]:
# Esto nos va a servir para más adelante:
label_names = dataset["train"].features["label"].names
label2id = {name: dataset["train"].features["label"].str2int(name) for name in label_names}
id2label = {id: label for label, id in label2id.items()}

print(label_names)
print(id2label[0], id2label[1])
print(label2id["neg"] , label2id["pos"])

## _Fine-tuning_ de BERT

Vamos a usar BERT para extraer una representación vectorial de cada secuencia y entrenar un clasificador lineal por encima. Entrenamos _toda_ la arquitectura en simultáneo en nuestros datos. Como partimos de pesos pre-entrenados, a esto se le llama **fine-tuning**.

Vamos a usar funciones de Hugging Face que van a automatizar muchas de las tareas que hicimos manualmente cuando usamos el modelo con embeddings estáticos word2vec.

Empezamos cargando el tokenizador y el modelo pre-entrenado de HF.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
bert_model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)
bert_model = bert_model.to(device)

**PREGUNTA 6**: ¿qué hace `.to(device)`?

El primer paso es la **tokenización**:

convertir cada ejemplo en una secuencia de tokens que el modelo pueda procesar. En particular, cada ejemplo queda representado como un diccionario del tipo `{'input_ids': ..., 'attention_mask': ..., 'label': ...}`.

In [ ]:
def tokenize_fn(examples):
    """Tokenización **sin aplicar padding** --> Lo aplicamos luego dinámicamente,
    en cada batch de entrenamiento
    """
    return tokenizer(
        examples["text"], truncation=True, max_length=tokenizer.model_max_length
    )

In [ ]:
# Ejemplo:
subset_example = dataset["train"][:3]
tokenized_subset = tokenize_fn(subset_example)

for k, v in tokenized_subset.items():
    print(k)
    print(v)
    print(len(v))
    print("Largo de cada input:", [len(x) for x in v])
    print("-"*70)

In [ ]:
tokenized_dataset = dataset.map(tokenize_fn)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
print(tokenized_dataset["train"][0])

In [ ]:
print(bert_model)

In [ ]:
param_names = [name for name, _ in bert_model.named_parameters()]
print(len(param_names))
print(param_names[:5])
print(param_names[-5:])

Vamos a hacer _fine-tuning_ de todos los pesos del modelo.

Alternativamente, podríamos entrenar la capa de clasificación y las últimas N capas de BERT, dejando las demás capas _congeladas_, corriendo esto:



In [ ]:
if False:
    # freeze todas las capas
    for param in bert_model.parameters():
        param.requires_grad = False
    # descongelar las ultimas 2 capas
    for param in bert_model.pre_classifier.parameters():
        param.requires_grad = True
    for param in bert_model.classifier.parameters():
        param.requires_grad = True
    # y los N ultimos transformer blocks:
    for param in bert_model.distilbert.transformer.layer[-2:].parameters():
        param.requires_grad = True

**PREGUNTA 7**: ¿qué quiere decir "congelar una capa"?

Usamos un **data collator** de HF que se encarga de agrupar los ejemplos en batches y hacer padding dinámicamente (esto es, padding solo hasta la longitud del ejemplo más largo en cada batch).

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Hacemos una función para evaluar métricas durante el entrenamiento.

In [ ]:
from transformers import EvalPrediction
from torch import nn

loss_fn = nn.CrossEntropyLoss()

def compute_metrics(logits, labels):
    """Args:
        logits: array shape (batch_size, num_labels)
        labels: array shape (batch_size,)
    """
    # Usamos torch para usar loss_fn, pero podriamos usar cpu y numpy
    if not isinstance(logits, torch.Tensor):
        logits = torch.tensor(logits)
    if not isinstance(labels, torch.Tensor):
        labels = torch.tensor(labels)
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == labels).float().mean().item()
    cross_entropy = loss_fn(logits, labels).item()
    return {"accuracy": accuracy, "cross_entropy": cross_entropy}

def compute_metrics_for_hf(pred: EvalPrediction) -> dict:
    """EvalPrediction: tupla con dos elementos: predictions y label_ids
    NOTE Trainer will put in EvalPrediction everything the model returns.
    """
    logits = pred.predictions
    labels = pred.label_ids
    return compute_metrics(logits, labels)

Para hacer el entrenamiento, usamos la clase `Trainer` de HF: funciona como un _wrapper_ que se encarga de hacer el loop de entrenamiento y evaluación que hicimos manualmente antes.

In [ ]:
n_epochs = 2
batch_size = 32
optimization_steps = int(np.ceil(len(tokenized_dataset["train"]) * n_epochs / batch_size))

print(f"N epochs: {n_epochs}")
print(f"Batch size: {batch_size}")
print(f"Optimization steps: {optimization_steps}")

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    "distilbert-ft-reviews",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    max_steps=optimization_steps,
    weight_decay=0.01,
    eval_strategy="steps",
    logging_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy", # el nombre de la metrica en compute_metrics()
    push_to_hub=False,
    seed=33,
    report_to="none",
)

trainer = Trainer(
    bert_model, args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics_for_hf,
)


In [ ]:
trainer.train()

In [ ]:
# Evaluar en test:
test_results = trainer.evaluate(tokenized_dataset["test"])

In [ ]:
print(test_results)

## BERT _pre-fine-tuned_

En lugar de hacer _fine-tuning_ de BERT en el dataset de reviews, podemos usar un modelo BERT que ya haya sido _fine-tuneado_ para resolver esta tarea, aunque sea en un dataset distinto.

Esto se conoce como "zero-shot", y es útil cuando no tenemos datos anotados para entrenar. Es como usar un modelo "en producción".

In [ ]:
from transformers import pipeline

sentiment_clf = pipeline(
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    device=device, batch_size=32
)

In [ ]:
# Inferencia en dataset de test:
from transformers.pipelines.pt_utils import KeyDataset

test_outputs = []
for output in sentiment_clf(KeyDataset(dataset["test"], "text"), top_k=None):
    test_outputs.append(output)

# Usamos KeyDataset para trabajar con el input que nos interesa como si fuera un
# dataset. Esto optimiza el cómputo. Ver:
# https://huggingface.co/docs/transformers/en/pipeline_tutorial#using-pipelines-on-a-dataset

In [ ]:
len(test_outputs)

In [ ]:
test_outputs[0]

Queremos los logits para calcular la pérdida. Para eso, cargamos el pipeline con el argumento `function_to_apply="none"`.



In [ ]:
sentiment_clf = pipeline(
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    device=device, batch_size=32, function_to_apply="none"
)

test_outputs = []
for output in sentiment_clf(KeyDataset(dataset["test"], "text"), top_k=None):
    test_outputs.append(output)

print(test_outputs[0])

In [ ]:
# Colocamos los logits en un np array (n_samples x num_classes)
test_logits = []

for output in test_outputs:
    logits = [0] * len(label_names)
    for item in output:
        label_ = item["label"][:3].lower()
        id_ = label2id[label_]
        logits[id_] = item["score"]
    logits_arr = np.array(logits)
    test_logits.append(logits_arr)

test_logits = np.vstack(test_logits)

In [ ]:
print(test_logits)
print(test_logits.shape)

In [ ]:
test_labels = dataset["test"]["label"]

print(test_labels[:5])
print(len(test_labels))

In [ ]:
compute_metrics(test_logits, test_labels)

## Análisis de errores

Suele ser útil hacer un análisis de errores de los modelos para detectar oportunidades de mejora, así como también errores en los datos.

En este caso, vamos a inspeccionar los falsos positivos y negativos _más groseros_ del primer modelo i.e. los ejemplos donde la pérdida es más alta.

In [ ]:
data_collator = trainer.data_collator

def run_inference(examples, model):
    """Agrega a un batch la proba, prediccion y loss de cada ejemplo de examples
    """
    examples = {k: v for k, v in examples.items() if k in ['label', 'input_ids', 'attention_mask']}
    batch = data_collator(examples)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    with torch.inference_mode():
        output = model(input_ids, attention_mask)
        batch["proba"] = torch.softmax(output.logits, dim=1)[:, 1]
        batch["predicted_label"] = torch.argmax(output.logits, axis=1)
    # reduction="none" --> loss por example
    loss = torch.nn.functional.cross_entropy(output.logits, labels, reduction="none")
    batch["loss"] = loss
    return batch

In [ ]:
# Ejemplo:
subset_example = tokenized_dataset["validation"][:3]
run_inference(subset_example, bert_model)

In [ ]:
bert_model.eval()
errors_dataset = tokenized_dataset['validation'].map(
    lambda examples: run_inference(examples, bert_model), batched=True, batch_size=32)

In [ ]:
errors_df = errors_dataset.to_pandas()[['text', 'label', 'proba', 'predicted_label', 'loss']]

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
# falsos positivos
errors_df.query("label == 0").sort_values("loss", ascending=False).head()

In [ ]:
# falsos negativos
errors_df.query("label == 1").sort_values("loss", ascending=False).head()

**PREGUNTA 8**: ¿cómo se interpreta el falso negativo más "fuerte" de la tabla anterior?